#  测试 llama.cpp http接口


In [58]:
import json;
import urllib.request;


def chat_model(prompt, model="qwen3:latest", url="http://192.168.9.179:11434/api/chat"):
    data = {
         "model": model,
        "messages": [
            {
                "role": "user", 
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0.0
        }
    };

    payload = json.dumps(data).encode("utf-8");
    request = urllib.request.Request(url, method="POST", data=payload);
    request.add_header("Content-Type", "application/json");
    content = "";
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode('utf-8');
            if not line:
                break;

            response_json = json.loads(line);
            # print(f"response_json:{response_json}");
            content +=response_json["message"]["content"];
        return content;
 


In [59]:
request = chat_model("您好， 你是？？" ); #, url="http://127.0.0.1:8899/v1/chat/completions" );

print(request);

您好！我是通义千问，是通义实验室研发的超大规模语言模型。我能够帮助您回答问题、创作文字、逻辑推理、编程、多语言理解等多种任务。如果您有任何问题或需要帮助，欢迎随时告诉我！😊


llama.cpp 自带的 HTTP 服务提供 OpenAI 兼容的 API，请求格式与调用 OpenAI 官方接口类似。以下是各核心端点的请求格式详解。

### 🔌 启动 HTTP 服务
启动服务（通常在 `localhost:8080`），这是后续所有请求的基础：
```bash
./llama-server --model models/你的模型.gguf
```

### 📝 核心 API 端点与请求格式
下表汇总了最常用的几个 HTTP API 端点：

| 端点 | 方法 | 主要功能 | 关键请求体/参数 | 主要返回字段 |
| :--- | :--- | :--- | :--- | :--- |
| **`/completion`** | POST | **文本补全** | `prompt`, `stream`, `temperature`, `max_tokens` | `content` |
| **`/v1/chat/completions`** | POST | **对话补全** (主流) | `model`, `messages`, `stream`, `temperature` | `choices[0].message.content` |
| **`/tokenize`** | POST | 文本转 Token ID | `content` | `tokens` |
| **`/embedding`** | POST | 获取文本向量 | `content` | `embedding` |
| **`/health`** | GET | 健康检查 | 无 | 服务状态信息 |

### 💬 详细请求示例

#### 1. 基础文本补全 (`/completion`)
这是较早期的端点，适用于简单的续写任务。
```bash
curl http://localhost:8080/completion -H "Content-Type: application/json" -d '{
  "prompt": "法国的首都是",
  "stream": false,
  "temperature": 0.8,
  "max_tokens": 50
}'
```

#### 2. 对话补全 (`/v1/chat/completions`)
**这是最常用、功能最丰富的端点**，其请求体格式完全遵循 OpenAI 标准。
```bash
curl http://localhost:8080/v1/chat/completions -H "Content-Type: application/json" -d '{
  "model": "gpt-3.5-turbo", // 模型名，可自定义
  "messages": [
    {"role": "system", "content": "你是一个乐于助人的助手。"},
    {"role": "user", "content": "请解释什么是知识蒸馏。"}
  ],
  "stream": false,
  "temperature": 0.7,
  "max_tokens": 500
}'
```

#### 3. 流式响应
将 `"stream": true`，数据会以 Server-Sent Events (SSE) 格式流式返回。
```bash
curl http://localhost:8080/v1/chat/completions -H "Content-Type: application/json" -d '{
  "model": "gpt-3.5-turbo",
  "messages": [{"role": "user", "content": "请逐字背诵《静夜思》"}],
  "stream": true
}'
```

#### 4. 函数调用（高级功能）
llama.cpp 支持函数调用（需使用支持此功能的模型并正确配置模板）。在 `/v1/chat/completions` 请求的 `tools` 参数中定义工具，模型会在需要时返回工具调用请求。
```bash
curl http://localhost:8080/v1/chat/completions -H "Content-Type: application/json" -d '{
  "model": "gpt-3.5-turbo",
  "messages": [{"role": "user", "content": "北京现在的天气怎么样？"}],
  "tools": [{
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "获取指定城市的当前天气",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "城市名"}
        },
        "required": ["location"]
      }
    }
  }]
}'
```
成功调用后，返回的 `message` 中会包含 `tool_calls` 字段，而非 `content`。

### 🛠️ 代码调用示例

#### Python (requests)
```python
import requests, json
url = "http://localhost:8080/v1/chat/completions"
headers = {"Content-Type": "application/json"}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [{"role": "user", "content": "你好"}],
    "stream": False
}
response = requests.post(url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2, ensure_ascii=False))
```

#### Python (OpenAI SDK)
llama.cpp 服务器与 OpenAI SDK 完全兼容。
```python
from openai import OpenAI
client = OpenAI(base_url="http://localhost:8080/v1", api_key="not-needed")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "你好"}]
)
print(response.choices[0].message.content)
```

### ✅ 关键提示与建议
1.  **模型与模板**：如需使用函数调用等高级功能，请务必选择**原生支持该功能的模型**（如 Llama 3.1、Qwen 2.5系列），并使用正确的 `--chat-template` 参数启动服务器。
2.  **内容类型**：所有请求头需包含 `Content-Type: application/json`。
3.  **参数调整**：核心参数 `temperature` (随机性)、`max_tokens` (生成长度)、`top_p` 等可根据需求调整。
4.  **解决超长输入**：输入过长可能导致错误，可检查服务器日志并根据提示调整 `--ctx-size` 启动参数。

如果你能告诉我你具体想用 llama.cpp 的 HTTP 接口来完成什么任务（例如，简单的问答对话、长文档处理，还是想实现类似AI Agent的函数调用），我可以提供更具体的参数配置建议或示例代码。